## Web Scraping 

https://consult.gov.scot/elections-and-constitutional-development-division/draft-referendum-bill/consultation/published_select_respondent

## Imports

In [39]:
# import sys
# print(sys.executable)
#! /opt/homebrew/Cellar/jupyterlab/3.4.8_1/libexec/bin/python3.11 -m pip install pandas

In [40]:
## Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Reading the file with the responses URLS

In [41]:
url_r_list = [line.strip() for line in open("../dataset/url_responses_list.txt", 'r')]
len(url_r_list)

6806

## Radio_Buttons Answers

In [62]:
radio_possible_answers={}
radio_possible_answers[0]="Individual"
radio_possible_answers[1]="Organisation"
radio_possible_answers[2]="Publish response with name"
radio_possible_answers[3]="Publish response only (without name)"
radio_possible_answers[4]="Do not publish response"


## Total questions in this consultations

In [63]:
questions_total=['What is your name?','Are you responding as an individual or an organisation?',
 'The Scottish Government would like your permission to publish your consultation response Please indicate your publishing preference:',
 'What are your views on the proposed arrangements for managing the referendum?',
 'What are your views on the proposed technical changes to polling and count arrangements?',
 'What are your views on the proposed changes to rules on permissible participants?']

In [153]:
responses_id=[]
answers_total=[]
for i in range(0, 500):
    url = url_r_list[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the question and answer elements using appropriate HTML tags and class names
    question_elements = soup.find_all('h3', class_= "dss-heading-x-small dss-heading-bold")


    question_elements = soup.find_all('h3', class_= "dss-heading-x-small dss-heading-bold")
    #print(question_elements)

    questions=[]
    for row in question_elements:
        questions.append(row.get_text().strip())
    for i in range(0, len(questions)):
        q= questions[i].strip().replace("\n                                                ","")                                          
        char_str = '' .join((z for z in q if not z.isdigit()))
        questions[i]=char_str.replace(".","")
    
    
    no_answers=[]
    for q in range(0, len(questions_total)):
        if questions_total[q] not in questions:
            no_answers.append(q)
    #print("No_answers %s" % no_answers)
    
    # response id
    li = soup.find('div', class_= "dss-easy-reading")
    child = li.findChildren("h2" , recursive=False)
    respond_id=child[0].get_text().split("Response ")[1]
    responses_id.append(respond_id)
    
    # answers
    answer_elements = soup.find_all('div', class_= "cs-question-padding")

    #1. answer.name 
    answers=[]
    answer_elements = soup.find_all('div', class_= "cs-fake-text-input")
    try:
        answers.append(answer_elements[0].get_text().strip())
    except:
        answers.append('Anonymous')
    #print(answers)
    
    #2. Radio Buttons: Indiviual/Organization; Publish info Answers
    radio_answer_elements = soup.find_all('span', class_= "sr-only")
    #for re in range(2, len(radio_answer_elements)):
    #    print("index is %s - index %s" %(re, radio_answer_elements[re]))
    radio_answers=[]
    first_row=0
    for row in range(2, len(radio_answer_elements)):
        if "Ticked" in radio_answer_elements[row].get_text().strip():
            if first_row == 0:
                index_rest=row
            radio_answers.append(row-index_rest)

    for ra in radio_answers:
         answers.append(radio_possible_answers[ra])
            
    # 4.5 Views    
    answers_elements = soup.find_all('div', class_= "cs-fake-textarea-input cs-fake-textarea-input-height-10")
    answers_tmp=[]
    for row in answers_elements:
        answers.append(row.get_text().strip())
    
  
    answers_final=[]

    a=0
    for k in range(1,7):
    
        if k in no_answers:
            answers_final.append("No Answered")
        else:
            #print(a)
            answers_final.append(answers[a])
            a=a+1
            
    
        
    answers_r=[respond_id] + answers_final
    answers_total.append(answers_r)


In [154]:
questions_total

['What is your name?',
 'Are you responding as an individual or an organisation?',
 'The Scottish Government would like your permission to publish your consultation response Please indicate your publishing preference:',
 'What are your views on the proposed arrangements for managing the referendum?',
 'What are your views on the proposed technical changes to polling and count arrangements?',
 'What are your views on the proposed changes to rules on permissible participants?']

In [155]:
header=["id"] + questions_total

In [156]:
#for i in range(0, len(answers_total)):
#    print(i, len(answers_total[i]))

## Creating a dataframe with the questions and answers

In [157]:
# Create the DataFrame
df = pd.DataFrame(answers_total, columns=header)

# Print the DataFrame
df

,id,What is your name?,Are you responding as an individual or an organisation?,The Scottish Government would like your permission to publish your consultation response Please indicate your publishing preference:,What are your views on the proposed arrangements for managing the referendum?,What are your views on the proposed technical changes to polling and count arrangements?,What are your views on the proposed changes to rules on permissible participants?
0,633076343,Fiona Smith,Individual,Individual,I have confidence in the polling arrangements ...,No Answered,The improvements seem sensible.
1,447847558,Anonymous,Individual,Individual,Why more uncertainty? We don't know what's goi...,I think this makes a lot of sense. It is the p...,This seems fine
2,1018825847,Anonymous,Individual,Individual,This appears to be satisfactory. There was a h...,Given the significant issues facing British ci...,No particular concerns
3,565446966,Craig Lynn,Individual,Individual,Proposals seem fair and abide by the normal co...,No Answered,"Appreciate the tightening of proxy votes, a hu..."
4,1050647939,Kieran Lennon,Individual,Individual,Words can't explain how much I want independen...,Fine with me.,Much better and more clearer and have a very g...
...,...,...,...,...,...,...,...
495,383726189,No need to lets the nats know,Individual,Individual,A disgrace. They say we will be taken out of t...,Seem fair.,Ok.
496,321003634,Suzanne hendry,Individual,Individual,Seems to be a well thought out process with em...,Again seems to be a well thought out process,All fine and good
497,1013545548,Anonymous,Individual,Individual,Agree,Agree,Agree
498,3270360,Anonymous,Individual,Individual,Keep all the postal votes in Scotland i dont s...,No Answered,I am Happy with all the above


In [158]:
#file_name="../dataset/consultation_referendum_bill.csv"
#df.to_csv(file_name)

In [160]:
html = df.to_html()

with open("file_name", "w", encoding="utf-8") as file:
    file.writelines('<meta charset="UTF-8">\n')
    file.write(html)